## PIthon - Ejemplo de integración de un programa Python con PI Data Archive y PI AF

Requiere tener instalado: pythonnet

http://pythonnet.github.io/

Debe correr en el servidor donde esté instalado el AF SDK.

In [4]:
# Error! No es esta la librería a instalar, es pip install pythonnet http://pythonnet.github.io/
# OJO: clr is a simple terminal string styling library. Its API is a port of the popular chalk module for javascript.
# https://pypi.org/project/clr/ Ejemplo: print(clr.red.bold('Hello world!')) NO HAY QUE INSTALAR CLR!!!

# Ejecutar lo siguiente en línea de comando
#!pip install pythonnet

In [1]:
# sys module provides access to some variables used or maintained by the interpreter and to functions that interact 
# strongly with the interpreter. It is always available. https://docs.python.org/3/library/sys.html
# clr es la forma de declarar el package pythonnet (.net para python)
import sys
import clr

In [2]:
sys.path.append(r'C:\Program Files (x86)\PIPC\AF\PublicAssemblies\4.0')  
clr.AddReference('OSIsoft.AFSDK')

In [3]:
from OSIsoft.AF import *
from OSIsoft.AF.PI import *
from OSIsoft.AF.Asset import *
from OSIsoft.AF.Data import *
from OSIsoft.AF.Time import *
from OSIsoft.AF.UnitsOfMeasure import *
from OSIsoft.AF.Search import *

In [4]:
#initializes a C# list: from System.Collections.Generic import List
# Implicit loading is deprecated. Please use clr.AddReference('System.Collections').
clr.AddReference('System.Collections')
from System.Collections.Generic import List

In [5]:
from datetime import datetime, timedelta

In [6]:
import pandas as pd

In [21]:
# Si coloco dir('OSIsoft.AF') no trae el detalle de las clases agregadas por este módulo. 
# Entendí que es por cómo se cargan en memoria las referencias a clases. Si sed indica dir() solamente, trae todo lo que 
# está cargado en memoria
#dir()

# Conexión a una AF Database

In [6]:
# PI Data Archive
#piServers = PIServers()  
#piServer = piServers.DefaultPIServer;
#print(piServer, 'Ok, pero no tengo permisos')

192.168.103.66


In [24]:
#piServers = PIServers()
#for piServer in piServers:
#    piServer.Connect()
#    print(piServer.Name + " -> " + str(piServer.ConnectionInfo.IsConnected))
#    piServer.Disconnect()

#No funciona porque no tengo permisos

In [5]:
afservers = PISystems()

In [8]:
#afserver = afservers.DefaultPISystem
AFServer = afservers['PARCCVWPIEDA01']
#AFServer = afservers['QARCCVWPIE01']
print(AFServer.Name)

PARCCVWPIEDA01


In [9]:
# Acceso a una base de datos que no sea la default:
DB = AFServer.Databases.DefaultDatabase
print(DB.Name)
DB = AFServer.Databases.get_Item("DOF")
print ('Database Name: {0}'.format(DB.Name))

DOF
Database Name: DOF


In [9]:
iterator = afservers.GetEnumerator()
for server in iterator:
    print(server.Name)

192.168.103.66
PARCCVWPIEDA01
QARCCVWPIE01


# Forma para recorrer la Estructura de elementos/assets

In [10]:
for element in DB.Elements:
    print(element.Name)

PAMPA ENERGIA S.A.


In [43]:
# Estas expresiones NO Funcionan
#elemTemp = DB.ElementTemplates['Instalacion-PMs'] o DB.ElementTemplates[0]
#element = DB.Elements.get_Item('El Mangrullo').Elements.get_Item('Pozos')

In [11]:
# El objeto iterable se debe asignar a una lista de python y esa lista sí se puede acceder:
element_lst = list(DB.Elements)
print(element_lst[0])

PAMPA ENERGIA S.A.


In [13]:
# 2° Nivel de elementos:
areas_lst = list(element_lst[0].Elements)

In [14]:
for element in DB.Elements:
    print(element.Name)
    for subelement in element.Elements:
        print(subelement.Name)
        #for attribute in subelement.Attributes:
        #    print(attribute.Name)

PAMPA ENERGIA S.A.
Aguaragüe
Anticlinal Campamento
Chirete
El Mangrullo
El Tordillo
Estación Fernández Oro
Gobernador Ayala
La Tapera-Puesto Quiroga
Parva Negra Este
Red de Transporte
Rincón de Aranda
Rincón del Mangrullo
Río Neuquén
Sierra Chata


In [25]:
for elementTemplate in DB.ElementTemplates:
    print(elementTemplate.Name)

Area
Compresor Plantas
Compresores (Grupo)
Facilidad
Instalaciones(Grupo)
Instalacion-Plantas
Instalacion-PMs
Medidores
Pozo
Pozo Parado
Pozos


In [49]:
element_templates_list = list(DB.ElementTemplates)
well_template = element_templates_list[8]
well_attribute_templates_list = list(well_template.AttributeTemplates)
print(well_attribute_templates_list[0])

% Gas Promedio Mes vs Cierre Mes Anterior


In [15]:
#elem_templates_lst = AFElementTemplatesList(DB.ElementTemplates)
#help(DB.ElementTemplates)
#help(well_lst[1].Attributes)
#help(AFAttributeList())

In [96]:
### SEGUIR BUSCANDO EN AFObject Class, porque los objetos heredan de esta clase el método GetItem
elem = DB.ElementTemplates.GetItem('Pozo', True, True, True)
print(elem.Name)

Pozo


In [16]:
#inspect.getargspec(AFElementTemplates.GetItem)

In [82]:
# module OSIsoft.AF.Asset
import inspect

for name, obj in inspect.getmembers('OSIsoft.AF.Asset'):
    if inspect.isclass(obj):
        print(obj)

<class 'str'>


In [86]:
#dir('OSIsoft.AF.Asset')
[m[0] for m in inspect.getmembers('OSIsoft.AF', inspect.isclass) if m[1].__module__ == 'OSIsoft.AF']

[]

# Busquedas de elementos

In [103]:
# Búsqueda de elementos que cumplen con un criterio (Template de Pozo) y se encuentran debajo de "El Mangrullo"
# AFElementSearch: Creates an instance of an AFElementSearch to search for AFElement objects using the specified 
# search query string. https://techsupport.osisoft.com/Documentation/PI-AF-SDK/Html/T_OSIsoft_AF_Search_AFElementSearch.htm
# FindElements() Performs a non-paged text search within the AFDatabase to retrieve a collection of AFElement objects 
# which have a field that matches the specified query string.

# Ejemplos: AFElementSearch(myDB, "FindValves", "Root:'WestPlant\Area32' (Category:'Valve' OR Name:Valve*)")
# AFEventFrameSearch(myDB, "FindEventFields", "Template:'Event' |Level:<90.0")
# search = AFElementSearch(DB, "busqueda", "Root:'PAMPA ENERGIA S.A.\Sierra Chata' Template:'Pozo'").FindElements()

In [17]:
pozos_lst = list(AFElementSearch(DB, "busqueda", "Root:'PAMPA ENERGIA S.A.\El Mangrullo' Template:'Pozo'").FindElements())
pozo_1 = pozos_lst[0]
attr = list(pozo_1.Attributes)
print(pozo_1.Name)
for attr in pozo_1.Attributes:
    print(attr.Name, attr.GetValue())

M.a-1013
Presión Boca de Pozo Promedio Hora 12,3499166996021
Prod. Gas Pozo Medido BP (RDBMS) 36,8456192016602
Fecha Max.Prod.Medida Gas 1555815600
Maxima Prod. Gas Medida Pozo 214,5
Presión Diferencial Diaria 0,122050285339355
Medicion Fecha 30/1/2022 00:00:00
GAP Presion Boca de Pozo Pt Created
GAP Presion Linea Pt Created
GAP Orificio 51
Venta de Gas STD (asignada con gas útil) 36,486008149508
Maxima Produccion Gas 220,474451096
Evento Año 2019
Pozo - Variacion Caudal Gas Medido 24hs -3,72820166761504
Caudal Pozo Alta Presion 0
Caudal Pozo Media Presion 0
Caudal Pozo Baja-Baja Presion 0
Prod. Promedio Venta Gas 9300 Año 3,73202855311355
Prod. Promedio Venta Gas 9300 Mes 33,9614598333333
Producción Acum. Venta Gas 9300 del Mes 1018,843795
Venta Asignada Balance Diario Gas 9300 40,459884
Prod. Promedio Gas Medido Mes 33,8347280292511
Factor Asignación Gas Promedio Mes 0,995012614218783
Prod. Promedio Petróleo Asignado Mes 0,862080366666666
Prod. Promedio Agua Asignado Mes 12,191063966

In [9]:
#AFValues = myAttribute.GetValues(myTimeRange, 0, Nothing)
attr = list(pozo_1.Attributes)
print(attr[0], "Pozo: ", attr[0].Element)

% Gas Promedio Mes vs Cierre Mes Anterior Pozo:  M.a-1013


# Obtención de Valores de Atributos

In [34]:
# Último valor de cada atributo para un pozo:
#--------------------------------------------
# Attribute Status: https://techsupport.osisoft.com/Documentation/PI-AF-SDK/html/T_OSIsoft_AF_Asset_AFValueStatus.htm
# Good = 192, Bad = 0, Questionable = 64, Subsituted = 24
attributes = AFAttributeList(well_lst[1].Attributes);
values = attributes.GetValue()
print('Pozo: ', well_lst[1])
#for val in values:
#    if val.IsGood:
#        print(val.Attribute, val.IsGood, val.Status, val.Timestamp, val.UOM, val.Value)
#    else:
#        print('---', val.Attribute, val.IsGood, val.Status, val.Timestamp, val.UOM, val.Value)

Pozo:  SCh.e-7


In [ ]:
# Often, however, there is a need to just access a few specific attributes from a collection of elements. 
# These examples use the LoadAttributes methods to load a specific list of attributes into the client. 
# https://techsupport.osisoft.com/Documentation/PI-AF-SDK/html/fc6c55c2-3879-4fe0-913c-3e759ba1b189.htm
# 1- Busco una lista de elementos:
well_lst = AFElementSearch(DB, "well_search", "Root:'PAMPA ENERGIA S.A.\Sierra Chata' Template:'Pozo'").FindElements()

In [37]:
# 2- Se debe armar la lista de templates de atributos (no es posible sólo pasar la lista de atributos)
# Para poder usar CollectionList en Python, se debe agregar esta librería: from System.Collections.Generic import List

In [ ]:
#elementlist = List[AFElement]()
#analysislist = List[AFAnalysis]()   #initializes a C# list
#search = AFAnalysisSearch(DB, 'All recalced analyses',"Category:'PMRecalc'").FindAnalyses()
#for analysis in search:
#    analysislist.Add(analysis)
#myAnalysisService.QueueCalculation(analysislist, timerange, myAnalysisService.CalculationMode.DeleteExistingData)

In [67]:
print(AFSearchField.Name)

1


In [66]:
well_template_lst = AFElementTemplate.FindElementTemplates(DB, "Template: 'Pozo'", AFSearchField.Name, AFSortField.Name, 
                                                           AFSortOrder.Ascending, 10000)

TypeError: No method matches given arguments for FindElementTemplates: (<class 'OSIsoft.AF.AFDatabase'>, <class 'str'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>)

In [52]:
# AFElementTemplate(string) es el constructor que instancia un nuevo objeto, en este caso, elementtemplate
well_template = AFElementTemplate('Pozo')
print(well_template.AttributeTemplates.Database) #Gets the collection of AFAttributeTemplate objects that are owned by this template.

for attr_temp in well_template.AttributeTemplates:
    print(attr_temp.Name)

AttributeError: 'AFDatabase' object has no attribute 'AFElementTemplate'

In [40]:
# Con GetItem puedo buscar por nombre los templates de atributos que necesito:
well_template.AttributeTemplates.GetItem('Caudal Instantaneo', True, True, True)

In [41]:
well_attribute_templates_List = List[AFAttributeTemplate]()   #initializes a C# list

In [38]:
#--------------------------------------------------------------------------------------------------------------
# 1- Necesito un objeto que sea el template de pozo:
# No encontré documentación de GetItem() no sé qué significan los True!
#--------------------------------------------------------------------------------------------------------------
well_template = DB.ElementTemplates.GetItem('Pozo', True, True, True)
print(well_template.Name)
# Pasé 3 días revisando la documentación de PI AF SDK y para Python no encontré la manera de armar una List de
# Probé y no funcionó:
# No se puede instanciar un objeto C# List con DB.ElementTemplates!!! element_template_lst = List(DB.ElementTemplates)
# AFElementTemplate.FindElementTemplates no funciona en Python 

# Otra forma, usando sólo Python sería crea una lista de python con el objeto DB.ElementTemplates y busco el objeto "Pozo"
#element_template_lst = list(DB.ElementTemplates)
# well_template = next((x for x in element_template_lst if x.Name == "Pozo"), None)

Pozo


# FUNCIONA!! Ojo la diferencia entre AFAttributeList y AFAttribute
Se usan distintos objetos AFData y AFDataList, los métodos de recuperación de datos son distintos

In [ ]:
# AFAttributeList --> AFListData
#AFListData.Summaries Method

### 1- Listado de Pozos

In [133]:
#for well in AFElementSearch(DB, "busqueda", "Root:'PAMPA ENERGIA S.A.\Sierra Chata' Template:'Pozo'").FindObjects():
#    #print(well.Name, type(well))
#    well_1 = well

In [18]:
# Armo un dict para acceder a los objetos por el nombre del objeto, en este caso, el nombre del pozo
wells_dict = dict()
for well in AFElementSearch(DB, "busqueda", "Root:'PAMPA ENERGIA S.A.\Sierra Chata' Template:'Pozo'").FindObjects():
    wells_dict[well.Name] = well
print(wells_dict['SCh-0012'].Name)

SCh-0012


In [23]:
# Cómo mostrar un slice de un dict:
dict(list(wells_dict.items())[0:5])

{'Ch-0002X': <OSIsoft.AF.Asset.AFElement object at 0x000001AFF6C6C220>,
 'SCh.e-7': <OSIsoft.AF.Asset.AFElement object at 0x000001AFF6C6C1F0>,
 'SCh.s-114': <OSIsoft.AF.Asset.AFElement object at 0x000001AFF6C6C280>,
 'SCh.x-1004(h)': <OSIsoft.AF.Asset.AFElement object at 0x000001AFF6C6C250>,
 'SCh.x-97': <OSIsoft.AF.Asset.AFElement object at 0x000001AFF6C6C190>}

### 2- Armo un listado de Atributos

In [27]:
attribute_lst = AFAttributeList()
attribute_lst.Add(wells_dict['SCh-0012'].Attributes.GetItem('Caudal Instantaneo', True, True, True))
attribute_lst.Add(wells_dict['SCh-0047'].Attributes.GetItem('Caudal Instantaneo', True, True, True))

In [24]:
# Definir el rango de tiempo

#myTimeRange.StartTime = New AFTime(DateTime.UtcNow.AddHours(-1))
#myTimeRange.EndTime = AFTime.Now
#### Tiempo Absoluto:
timerange_start = AFTime('2021-02-02 00:00:00')
timerange_span = AFTimeSpan.Parse('1d')
timerange_end = AFTime.op_Addition(timerange_start, timerange_span)
timerange = AFTimeRange(timerange_start, timerange_end)
print(timerange_start, timerange_end, timerange_span)

#### Tiempo Relativo:
#timerange = AFTimeRange("*-31d", "*-30d")
#print(type(timerange), timerange.StartTime, timerange.EndTime, timerange.Span) # Span parece que es en días.

2/2/2021 00:00:00 3/2/2021 00:00:00 1d


In [35]:
# The PIPagingConfiguration is used to describe how PIPointList and AFListData data access calls should page results 
# for better performance.
# PIPagingConfiguration(PIPageType pageType, int pageSize)
# The PIPageType is used to determine how partial results from list data access calls should be grouped while being 
# returned to the client. Choosing the appropriate partition type will improve performance:
# TagCount 0: Results are paged from the PI Data Archive by tag count, with a maximum tags per page specified in the 
# PageSize property.
# EventCount 2: Results are paged from the PI Data Archive by event count. A new page of data is retrieved when the number 
# of events exceeds the PageSize property. All events for a single tag are retrieved in one page.
pagingconfig = PIPagingConfiguration(PIPageType.TagCount, 100)

In [39]:
# RecordedValues(AFTimeRange timeRange, AFBoundaryType boundaryType, string filterExpression, bool includeFilteredValues,
# PIPagingConfiguration pagingConfig, int maxCount = 0)
values_lst = attribute_lst.Data.RecordedValues(timerange, AFBoundaryType.Inside, "", False, pagingconfig, 50)
# trae sólo 50 registros

In [40]:
for values in values_lst:
    print(values.Attribute.Element, values.Attribute, values.Attribute.DisplayUOM)
    att_UOM = values.Attribute.DisplayUOM
    for value in values:
        print(value.Timestamp, value.Value, att_UOM)

SCh-0012 Caudal Instantaneo Miles Metros Cubicos
2/2/2021 00:00:09 43.50760269165039 Miles Metros Cubicos
2/2/2021 00:00:39 40.528831481933594 Miles Metros Cubicos
2/2/2021 00:01:09 39.27621841430664 Miles Metros Cubicos
2/2/2021 00:01:39 39.798763275146484 Miles Metros Cubicos
2/2/2021 00:02:09 42.133182525634766 Miles Metros Cubicos
2/2/2021 00:02:39 42.6055793762207 Miles Metros Cubicos
2/2/2021 00:04:39 42.57516098022461 Miles Metros Cubicos
2/2/2021 00:05:09 39.96065139770508 Miles Metros Cubicos
2/2/2021 00:05:39 41.9393196105957 Miles Metros Cubicos
2/2/2021 00:06:09 42.97857666015625 Miles Metros Cubicos
2/2/2021 00:07:09 42.24734115600586 Miles Metros Cubicos
2/2/2021 00:07:39 41.15255355834961 Miles Metros Cubicos
2/2/2021 00:08:09 41.086875915527344 Miles Metros Cubicos
2/2/2021 00:08:39 42.711185455322266 Miles Metros Cubicos
2/2/2021 00:09:09 39.64622497558594 Miles Metros Cubicos
2/2/2021 00:09:39 41.85784912109375 Miles Metros Cubicos
2/2/2021 00:10:09 42.7234992980957 M

#### Summary Values

In [43]:
# A AFTimeSpan represents a time interval (duration of time or elapsed time) that is measured as a positive or negative 
# number of years, months, days, hours, minutes, seconds, milliseconds, and fractions of a millisecond. 
# Create AFTimeSpan using constructor
now = AFTime.Now
span = AFTimeSpan.Parse('1h')
print("TimeSpan ", span)

TimeSpan  1hr


In [47]:
# AFSummaryTypes: Total, Average, Maximun, Minimum, Range (diferencia entre Max y Min), StdDev, etc 
# https://techsupport.osisoft.com/Documentation/PI-AF-SDK/Html/T_OSIsoft_AF_Data_AFSummaryTypes.htm
# AFCalculationBasis: 
# TimeWeighted = Weight the values in the calculation by the time over which they apply. 
# Interpolation is based on whether the attribute is stepped. 
# Interpolated events are generated at the boundaries if necessary.
# EventWeighted: Evaluate values with equal weighting for each event. 
# No interpolation is done except in the case of expression calculations evaluated by the PI Server. 
# There must be at least one event within the time range to perform a successful calculation. 
# Two events are required for standard deviation. 
# In handling events at the boundary of the calculation, the AF SDK uses following rules:
# 1) use events at both boundaries when there is only one calculation interval;
# 2) include events at start time in multiple intervals and the intervals are in ascending time order;
# 3) include events at the end time in multiple intervals and the intervals are in descending time order.

summary_type = AFSummaryTypes.Average
summ_values_dict = attribute_lst.Data.Summaries(timerange, span, summary_type, AFCalculationBasis.TimeWeighted, 
                                          AFTimestampCalculation.Auto, pagingconfig)

In [32]:
# NOTA: el objeto summ_values_dict es un diccionario (C#) que se accede por la key AFSummaryTypes indicada en Data.Summaries
# y estaría formado por Key=AFSummaryType usada, AFValues

In [49]:
for summ in summ_values_dict:
    print(summ[summary_type].Attribute.Element, summ[summary_type].Attribute)
    for val in summ[summary_type]:
        print(val.Timestamp, val.IsGood, val.Value)

SCh-0012 Caudal Instantaneo
2/2/2021 00:00:00 True 25.931683794083195
2/2/2021 01:00:00 True 25.88488546790946
2/2/2021 02:00:00 True 26.03248329165818
2/2/2021 03:00:00 True 25.812794734930815
2/2/2021 04:00:00 True 25.92428486696973
2/2/2021 05:00:00 True 25.891272480769064
2/2/2021 06:00:00 True 26.01278297662877
2/2/2021 07:00:00 True 25.959127117305734
2/2/2021 08:00:00 True 25.79951366744703
2/2/2021 09:00:00 True 25.917883201753952
2/2/2021 10:00:00 True 25.92630001989274
2/2/2021 11:00:00 True 26.203477068853154
2/2/2021 12:00:00 True 26.496124450120792
2/2/2021 13:00:00 True 26.26444714823118
2/2/2021 14:00:00 True 26.41511463696335
2/2/2021 15:00:00 True 26.253248197272768
2/2/2021 16:00:00 True 26.360675940085752
2/2/2021 17:00:00 True 26.039082352296727
2/2/2021 18:00:00 True 25.996902315163112
2/2/2021 19:00:00 True 26.09127089971513
2/2/2021 20:00:00 True 25.913525241528124
2/2/2021 21:00:00 True 25.985483416696006
2/2/2021 22:00:00 True 26.14728404396298
2/2/2021 23:00:0

# ESTE TRAE PARA UN LISTADO DE POZOS Y ATRIBUTOS LOS VALORES HORARIOS

#### Recorded Values At Times

In [61]:
# The PIPagingConfiguration is used to describe how PIPointList and AFListData data access calls should page results 
# for better performance.
# PIPagingConfiguration(PIPageType pageType, int pageSize)
# The PIPageType is used to determine how partial results from list data access calls should be grouped while being 
# returned to the client. Choosing the appropriate partition type will improve performance:
# TagCount 0: Results are paged from the PI Data Archive by tag count, with a maximum tags per page specified in the 
# PageSize property.
# EventCount 2: Results are paged from the PI Data Archive by event count. A new page of data is retrieved when the number 
# of events exceeds the PageSize property. All events for a single tag are retrieved in one page.
pagingconfig = PIPagingConfiguration(PIPageType.TagCount, 100)

In [125]:
attribute_lst = AFAttributeList()
dataset = list()
for well in wells_dict:
    attribute_lst.Add(wells_dict[well].Attributes.GetItem('Caudal Instantaneo', True, True, True))
    attribute_lst.Add(wells_dict[well].Attributes.GetItem('Presión Boca de Pozo', True, True, True))

#attribute_lst.Add(wells_dict['SCh-0047'].Attributes.GetItem('Caudal Instantaneo', True, True, True))
#attribute_lst.Add(wells_dict['SCh-0001X'].Attributes.GetItem('Caudal Instantaneo', True, True, True))

In [126]:
# Armo un LIST (C#) de AFTime objects
#### Necesito el timesrange de datos. Lo defino como Tiempo Absoluto:
timerange_start = AFTime('2021-02-02 00:00:00')
timerange_span = AFTimeSpan.Parse('1d')
timerange_end = AFTime.op_Addition(timerange_start, timerange_span)
timerange = AFTimeRange(timerange_start, timerange_end)

times = List[AFTime]()
time_span = AFTimeSpan.Parse('1h')
time_aux = timerange_start
times.Add(time_aux)
for h in range(1, int(timerange.Span.TotalHours)):
    time_aux = AFTime.op_Addition(time_aux, time_span)
    times.Add(time_aux)

#for i in times:
#    print(i)

#### Cómo parsear un timerange en sus partes, year, month, day, etc

In [79]:
timerange_start = AFTime('2021-02-02 00:00:00')
timerange_span = AFTimeSpan.Parse('1h')
timerange_end = AFTime.op_Addition(timerange_start, timerange_span)
timerange = AFTimeRange(timerange_start, timerange_end)
print( timerange.Span.ToString(), timerange.Span.Hours, timerange.Span.Days, timerange.Span.TotalHours)

01:00:00 1 0 1.0


In [84]:
timerange_start = AFTime('2021-02-02 00:00:00')
timerange_span = AFTimeSpan.Parse('1d')
timerange_end = AFTime.op_Addition(timerange_start, timerange_span)
timerange = AFTimeRange(timerange_start, timerange_end)
print( timerange.Span.ToString(), timerange.Span.Hours, timerange.Span.Days, timerange.Span.TotalHours,
      timerange.Span.Minutes, timerange.Span.TotalMinutes)

1.00:00:00 0 1 24.0 0 1440.0


#### Cómo obtener la hora de Now() para realizar los cálculos justo en la hora

In [107]:
date_str = datetime(AFTime.Now.LocalTime.Year, AFTime.Now.LocalTime.Month, AFTime.Now.LocalTime.Day, AFTime.Now.LocalTime.Hour, 0, 0)
print(date_str)

2021-03-05 15:00:00


In [127]:
# IEnumerable<AFValues> InterpolatedValuesAtTimes(IList<AFTime> times, string filterExpression, bool includeFilteredValues,
# PIPagingConfiguration pagingConfig)

# IEnumerable<AFValues> RecordedValuesAtTimes(IList<AFTime> times, AFRetrievalMode mode, PIPagingConfiguration pagingConfig)
# AFRetrievalMode: Auto 0:Automatically determine the best retrieval mode.
# AtOrBefore 1: Return a recorded value at the passed time or if no value exists at that time, the previous recorded value.
# Before 6: Return the first recorded value before the passed time.
# AtOrAfter 2: Return a recorded value at the passed time or if no value exists at that time, the next recorded value.
# After 7: Return the first recorded value after the passed time.
# Exact 4: Return a recorded value at the passed time or return an error if none exists.
recorded_values_lst = attribute_lst.Data.RecordedValuesAtTimes(times, AFRetrievalMode.AtOrBefore, pagingconfig)

In [172]:
for values in recorded_values_lst:
    if values.Attribute.Element.Name == 'SCh-93':
        print(values.Attribute.Element, values.Attribute)
        for val in values:
            if val.IsGood:
                date_str = datetime(val.Timestamp.LocalTime.Year, val.Timestamp.LocalTime.Month, val.Timestamp.LocalTime.Day, 
                            val.Timestamp.LocalTime.Hour, 0, 0)
                timestamp = datetime(val.Timestamp.LocalTime.Year, val.Timestamp.LocalTime.Month, val.Timestamp.LocalTime.Day, 
                            val.Timestamp.LocalTime.Hour, val.Timestamp.LocalTime.Minute, val.Timestamp.LocalTime.Second)
                td = timestamp-date_str
                if td.seconds > 0:
                    #print(date_str, timestamp, td.seconds)
                    date_str = date_str + add_1_hour
                print(val.Timestamp, date_str, val.Value)

SCh-93 Caudal Instantaneo
1/2/2021 23:59:59 2021-02-02 00:00:00 24.42092514038086 False
2/2/2021 00:59:59 2021-02-02 01:00:00 24.930519104003906 False
2/2/2021 01:59:59 2021-02-02 02:00:00 25.750423431396484 False
2/2/2021 02:58:29 2021-02-02 03:00:00 25.40715217590332 False
2/2/2021 03:58:59 2021-02-02 04:00:00 25.941129684448242 False
2/2/2021 04:59:59 2021-02-02 05:00:00 24.89389419555664 False
2/2/2021 05:59:59 2021-02-02 06:00:00 24.6317138671875 False
2/2/2021 06:59:59 2021-02-02 07:00:00 25.371322631835938 False
2/2/2021 07:59:59 2021-02-02 08:00:00 24.714937210083008 False
2/2/2021 08:59:59 2021-02-02 09:00:00 25.568994522094727 False
2/2/2021 09:16:59 2021-02-02 10:00:00 0.0 False
2/2/2021 10:56:54 2021-02-02 11:00:00 0.0 False
2/2/2021 10:56:54 2021-02-02 11:00:00 0.0 False
2/2/2021 10:56:54 2021-02-02 11:00:00 0.0 False
2/2/2021 13:05:29 2021-02-02 14:00:00 0.0 False
2/2/2021 14:55:29 2021-02-02 15:00:00 0.0 False
2/2/2021 15:59:59 2021-02-02 16:00:00 41.193641662597656 Fals

In [173]:
add_1_hour = timedelta(hours=1)

dataset_1 = list()

for values in recorded_values_lst:
    #print(values.Attribute.Element, values.Attribute)
    for val in values:
        if val.IsGood:
            # Como el criterio es que traiga el valor de la hora o el inmediato anterior, 
            # hay que sumarle una hora al timestamp sólo en caso de que no sea la hora exacta.
            # El dataset quedará con horas repetidas porque en los casos en que tiene un valor para una hora y no 
            # tiene en las siguientes horas, en las siguientes repetirá el valor y su timestamp
            # habría que luego hacer un unique con el dataframe
            date_str = datetime(val.Timestamp.LocalTime.Year, val.Timestamp.LocalTime.Month, val.Timestamp.LocalTime.Day, 
                            val.Timestamp.LocalTime.Hour, 0, 0)
            timestamp = datetime(val.Timestamp.LocalTime.Year, val.Timestamp.LocalTime.Month, val.Timestamp.LocalTime.Day, 
                            val.Timestamp.LocalTime.Hour, val.Timestamp.LocalTime.Minute, val.Timestamp.LocalTime.Second)
            td = timestamp-date_str
            if td.seconds > 0:
                #print(date_str, timestamp, td.seconds)
                date_str = date_str + add_1_hour
            # print(val.Timestamp, date_str, print_value)
            dataset_1.append([values.Attribute.Element.Name, values.Attribute.Name, date_str, val.Value])

In [174]:
df = pd.DataFrame(dataset, columns=['Well', 'Attribute', 'Timestamp', 'Value'])
df

,Well,Attribute,Timestamp,Value
0,SCh.e-7,Caudal Instantaneo,2021-02-02 00:00:00,25.966761
1,SCh.e-7,Caudal Instantaneo,2021-02-02 01:00:00,25.149343
2,SCh.e-7,Caudal Instantaneo,2021-02-02 02:00:00,26.740486
3,SCh.e-7,Caudal Instantaneo,2021-02-02 03:00:00,26.640495
4,SCh.e-7,Caudal Instantaneo,2021-02-02 04:00:00,26.604902
...,...,...,...,...
2496,SChS-0001X,Presión Boca de Pozo,2021-02-02 19:00:00,6.995497
2497,SChS-0001X,Presión Boca de Pozo,2021-02-02 20:00:00,7.038414
2498,SChS-0001X,Presión Boca de Pozo,2021-02-02 21:00:00,6.995497
2499,SChS-0001X,Presión Boca de Pozo,2021-02-02 22:00:00,7.081331


In [166]:
import pathlib
from pathlib import Path

import os
import os.path

#path = os.path.join(pathlib.Path().absolute(), '1-Inputs')
#path = os.path(pathlib.Path().absolute())
#p = Path(path)

df.to_csv(os.path.join(pathlib.Path().absolute(), 'prueba.csv'), encoding='ISO-8859-1',sep=';', header=False, index=False)
df.to_csv(os.path.join(pathlib.Path().absolute(), 'prueba2.csv'), encoding='utf-8',sep=';', header=False, index=False, float_format="%.8f")

#### Interpolated Values

In [ ]:
# Definir el rango de tiempo
#### Tiempo Absoluto:
timerange_start = AFTime('2021-02-02 00:00:00')
timerange_span = AFTimeSpan.Parse('1d')
timerange_end = AFTime.op_Addition(timerange_start, timerange_span)
timerange = AFTimeRange(timerange_start, timerange_end)
print(timerange_start, timerange_end, timerange_span)

#### Tiempo Relativo:
#timerange = AFTimeRange("*-31d", "*-30d")
#print(type(timerange), timerange.StartTime, timerange.EndTime, timerange.Span) # Span parece que es en días.

In [181]:
# Definir el intervalo de consulta:

# A AFTimeSpan represents a time interval (duration of time or elapsed time) that is measured as a positive or negative 
# number of years, months, days, hours, minutes, seconds, milliseconds, and fractions of a millisecond. 
# Create AFTimeSpan using constructor

now = AFTime.Now
span = AFTimeSpan.Parse('-1h')
print("TimeSpan ", span)

TimeSpan  -1hr


In [182]:
# Realizar la consulta:

# When a positive interval (span) is specified, the interval calculation begins at the earliest bounding time in the 
# timeRange and applies the interval repeatedly in time ascending direction to generate the calculation intervals.
# If a negative interval is specified, the interval calculation begins at the latest bounding time in the timeRange and 
# applies the interval repeatedly in time descending direction to generate the calculation intervals. Note that the order of values returned will still be reflected by the timeRange, regardless of the interval sign.

attribute = attribute_lst.GetItem('Caudal Instantaneo', True, True, True)
values_lst = attribute.Data.InterpolatedValues(timerange, span, att_UOM, "", False)

In [183]:
# Valores:

print('Elemento:', attribute.Element, 'Attribute:', attribute, 'Span:', span, 'Time Range:', timerange)
for val in values_lst:
    print(type(val), val.Timestamp, val.IsGood, val.Value)

Elemento: SCh-0012 Attribute: Caudal Instantaneo Span: -1hr Time Range: 2/2/2021 00:00:00 - 3/2/2021 00:00:00
<class 'OSIsoft.AF.Asset.AFValue'> 2/2/2021 00:00:00 True 43.26295471191406
<class 'OSIsoft.AF.Asset.AFValue'> 2/2/2021 01:00:00 True 41.85419464111328
<class 'OSIsoft.AF.Asset.AFValue'> 2/2/2021 02:00:00 True 41.22639465332031
<class 'OSIsoft.AF.Asset.AFValue'> 2/2/2021 03:00:00 True 42.318023681640625
<class 'OSIsoft.AF.Asset.AFValue'> 2/2/2021 04:00:00 True 40.243228912353516
<class 'OSIsoft.AF.Asset.AFValue'> 2/2/2021 05:00:00 True 40.53416061401367
<class 'OSIsoft.AF.Asset.AFValue'> 2/2/2021 06:00:00 True 42.32358169555664
<class 'OSIsoft.AF.Asset.AFValue'> 2/2/2021 07:00:00 True 40.65400314331055
<class 'OSIsoft.AF.Asset.AFValue'> 2/2/2021 08:00:00 True 43.14912796020508
<class 'OSIsoft.AF.Asset.AFValue'> 2/2/2021 09:00:00 True 41.234352111816406
<class 'OSIsoft.AF.Asset.AFValue'> 2/2/2021 10:00:00 True 41.20669937133789
<class 'OSIsoft.AF.Asset.AFValue'> 2/2/2021 11:00:0

#### Recorded Values
No sirve porque trae todos los valores guardados en el intervalo

AFValues RecordedValues( <br>
> AFTimeRange timeRange, <br>
> AFBoundaryType boundaryType, <br>
> UOM desiredUOM (OSIsoft.AF.UnitsOfMeasure.UOM),<br>
> string filterExpression, <br>
> bool includeFilteredValues, <br>
> int maxCount = 0 <br>
)

In [ ]:
# Definir el rango de tiempo
#### Tiempo Absoluto:
timerange_start = AFTime('2021-02-02 00:00:00')
timerange_span = AFTimeSpan.Parse('1d')
timerange_end = AFTime.op_Addition(timerange_start, timerange_span)
timerange = AFTimeRange(timerange_start, timerange_end)
print(timerange_start, timerange_end, timerange_span)

#### Tiempo Relativo:
#timerange = AFTimeRange("*-31d", "*-30d")
#print(type(timerange), timerange.StartTime, timerange.EndTime, timerange.Span) # Span parece que es en días.

In [198]:
# Consulta: OJO estos parámetros funcionan sólo para cuando consulto un atributo 
# (son distintos si consulto un attributeList)
values_lst = attribute.Data.RecordedValues(timerange, AFBoundaryType.Inside, attribute.DisplayUOM, "", False, 24)

In [197]:
# Valores:
print('Elemento:', attribute.Element, 'Attribute:', 'Time Range:', timerange)
for val in values_lst:
    print(type(val), val.Timestamp, val.IsGood, val.Value)

Elemento: SCh-0012 Attribute: Time Range: 2/2/2021 00:00:00 - 3/2/2021 00:00:00
<class 'OSIsoft.AF.Asset.AFValue'> 2/2/2021 00:00:09 True 43.50760269165039
<class 'OSIsoft.AF.Asset.AFValue'> 2/2/2021 00:00:39 True 40.528831481933594
<class 'OSIsoft.AF.Asset.AFValue'> 2/2/2021 00:01:09 True 39.27621841430664
<class 'OSIsoft.AF.Asset.AFValue'> 2/2/2021 00:01:39 True 39.798763275146484
<class 'OSIsoft.AF.Asset.AFValue'> 2/2/2021 00:02:09 True 42.133182525634766
<class 'OSIsoft.AF.Asset.AFValue'> 2/2/2021 00:02:39 True 42.6055793762207
<class 'OSIsoft.AF.Asset.AFValue'> 2/2/2021 00:04:39 True 42.57516098022461
<class 'OSIsoft.AF.Asset.AFValue'> 2/2/2021 00:05:09 True 39.96065139770508
<class 'OSIsoft.AF.Asset.AFValue'> 2/2/2021 00:05:39 True 41.9393196105957
<class 'OSIsoft.AF.Asset.AFValue'> 2/2/2021 00:06:09 True 42.97857666015625
<class 'OSIsoft.AF.Asset.AFValue'> 2/2/2021 00:07:09 True 42.24734115600586
<class 'OSIsoft.AF.Asset.AFValue'> 2/2/2021 00:07:39 True 41.15255355834961
<class 

#### Plot Values

In [190]:
# Retrieves values over the specified time range suitable for plotting over the number of intervals 
# (typically represents pixels).
# AFValues PlotValues(AFTimeRange timeRange, int intervals (The number of intervals to plot over. 
# Typically, this would be the number of horizontal pixels in the trend.), UOM desiredUOM)
# Nota: cuanto más grande el parámetro "intervals" más datos trae...

values_lst = attribute.Data.PlotValues(timerange, 6, att_UOM)

In [189]:
# Valores:

print('Elemento:', attribute.Element, 'Attribute:', attribute, 'Span:', span, 'Time Range:', timerange)
for val in values_lst:
    print(type(val), val.Timestamp, val.IsGood, val.Value)

Elemento: SCh-0012 Attribute: Caudal Instantaneo Span: -1hr Time Range: 2/2/2021 00:00:00 - 3/2/2021 00:00:00
<class 'OSIsoft.AF.Asset.AFValue'> 2/2/2021 00:00:00 True 43.26295471191406
<class 'OSIsoft.AF.Asset.AFValue'> 2/2/2021 00:45:39 True 36.6896858215332
<class 'OSIsoft.AF.Asset.AFValue'> 2/2/2021 01:52:09 True 43.8211784362793
<class 'OSIsoft.AF.Asset.AFValue'> 2/2/2021 01:59:09 True 39.421932220458984
<class 'OSIsoft.AF.Asset.AFValue'> 2/2/2021 02:00:09 True 41.56065368652344
<class 'OSIsoft.AF.Asset.AFValue'> 2/2/2021 02:16:09 True 44.12886428833008
<class 'OSIsoft.AF.Asset.AFValue'> 2/2/2021 03:40:39 True 36.00777816772461
<class 'OSIsoft.AF.Asset.AFValue'> 2/2/2021 03:59:39 True 41.404964447021484
<class 'OSIsoft.AF.Asset.AFValue'> 2/2/2021 04:00:09 True 39.70982360839844
<class 'OSIsoft.AF.Asset.AFValue'> 2/2/2021 05:16:09 True 43.47256088256836
<class 'OSIsoft.AF.Asset.AFValue'> 2/2/2021 05:25:39 False Comm Fail
<class 'OSIsoft.AF.Asset.AFValue'> 2/2/2021 05:31:19 True 37.

In [26]:
# 2- para luego armar una colección con los atributos de presión, temp que necesito
# AFAttributeTemplate.FindAttributeTemplate Method (AFElementTemplate, Guid)
# AFAttribute.FindAttribute Method (AFBaseElement, AFAttributeTemplate)


In [27]:
#well_element_template = DB.ElementTemplates.FindElementTemplates(DB, 'Pozo', AFSearchField.Name, AFSortField.Name, AFSortOrder.Ascending, 1)
well_element_template = AFElementTemplate.FindElementTemplates(DB, "Pozo")#, AFSearchField.Name, AFSortField.Name, AFSortOrder.Ascending, 1)

TypeError: No method matches given arguments for FindElementTemplates: (<class 'OSIsoft.AF.AFDatabase'>, <class 'str'>)

In [12]:
# requiere la sentencia clr.AddReference('System.Collections')
well_attr_lst = List[AFAttributeTemplate]()

In [25]:
for elementTemplate in DB.ElementTemplates:
    print(elementTemplate.Name)

Area
Compresor Plantas
Compresores (Grupo)
Facilidad
Instalaciones(Grupo)
Instalacion-Plantas
Instalacion-PMs
Medidores
Pozo
Pozo Parado
Pozos


# Armado de una Estructura Dict con los datos recorded

In [8]:
# 1- Obtengo los pozos:
well_lst = list(AFElementSearch(DB, "well_search", "Root:'PAMPA ENERGIA S.A.\Sierra Chata' Template:'Pozo'").FindElements())

In [26]:
# 2-Armo un dict para acceder a los objetos por el nombre del objeto, en este caso, el nombre del pozo
well_dict = dict()
for i in well_lst:
    well_dict[i.Name] = i
print(well_dict['Ch-0002X'].Attributes)
print(well_lst[0])
print(well_dict[well_lst[0].Name].Attributes)

Attributes
Ch-0002X
Attributes


In [ ]:
piPoint = AFAttribute.FindAttribute("string_to_attribute", piSys)  
recordedValues = piPoint.GetValues(nowTime, 0, None)
 '#create a dictionary entry for Python '
 recordedValuesDict = dict()
 for event in recordedValues:
    dt = datetime.datetime.strptime(event.Timestamp.LocalTime.ToString(),'%d.%m.%Y %H:%M:%S')
    recordedValuesDict[dt] = event.Value
 
#load the data
df = pd.DataFrame(recordedValuesDict.items(), columns=['TimeStamp', 'Value'])
 #Send it to a dateTime Index then set the index
df['TimeStamp'] = pd.to_datetime(df['TimeStamp'])


# Acceso a Valores

In [10]:
# This property returns a temporary AFData object to be used for accessing data associated with this AFAttribute.
# and is used to retrieve and set extended historical data.
attr[0].Data

## Current Value

In [45]:
# No existe el currentvalue para un atributo:
# print(attr[0].Data.CurrentValue())
# AFAttribute.GetValue Method

# Returns the AFValue for the attribute. If AFValue.IsGood returned by the data reference is true, then the returned value 
# is converted to the proper UOM and Type.
print(attr[0].GetValue())

-11,4409035383812


In [51]:
# AFAttribute.GetValue Method (AFTime)
# AFTime: used to represent time and convert between various time formats.
oneHourAgo = AFTime("*-1h")
print(oneHourAgo)
now = AFTime('*')
print('Attr: ', attr[0].Name)
print('Now: ', attr[0].GetValue(now))
print('1 Hour Ago: ', attr[0].GetValue(oneHourAgo))

27/2/2021 18:56:40
Attr:  % Gas Promedio Mes vs Cierre Mes Anterior
Now:  -11,4453750688756
1 Hour Ago:  -11,4391450885459


In [35]:
#myTimeRange.StartTime = New AFTime(DateTime.UtcNow.AddHours(-1))
#myTimeRange.EndTime = AFTime.Now
timerange = AFTimeRange("*-1d", "*")
print(type(timerange))

<class 'OSIsoft.AF.Time.AFTimeRange'>


In [36]:
# A AFTimeSpan represents a time interval (duration of time or elapsed time) that is measured as a positive or negative 
# number of years, months, days, hours, minutes, seconds, milliseconds, and fractions of a millisecond. 
# Create AFTimeSpan using constructor
now = AFTime.Now
span = AFTimeSpan.Parse('1h')
#span = AFTimeSpan(hours=12, minutes=30)
newTime = span.Add(now)
print("TimeSpan = {0}, Now+Span = {1}", span, newTime.LocalTime)

TimeSpan = {0}, Now+Span = {1} 1hr 27/2/2021 17:47:57


## Valores Interpolados

In [37]:
# https://techsupport.osisoft.com/Documentation/PI-AF-SDK/html/M_OSIsoft_AF_Data_AFData_InterpolatedValues.htm
# InterpolatedValues: Retrieves interpolated values over the specified time range at the specified sampling interval.
# Returns anAFValue object
# InterpolatedValues(AFTimeRange timeRange, AFTimeSpan interval, UOM desiredUOM, string filterExpression, bool includeFilteredValues)
# interval: Type: OSIsoft.AF.Time.AFTimeSpan The Sample interval. If specified in hours, minutes, seconds, or milliseconds,
interpolated_values = list(attr[0].Data.InterpolatedValues(timerange, span, None, "", False))
for i in range(0, len(interpolated_values)-1):
    print(interpolated_values[i].Value, interpolated_values[i].Timestamp)

-11.395340572190262 26/2/2021 16:47:55
-11.396839051876787 26/2/2021 17:47:55
-11.398626745497783 26/2/2021 18:47:55
-11.401392699367001 26/2/2021 19:47:55
-11.404158653236209 26/2/2021 20:47:55
-11.406924607105427 26/2/2021 21:47:55
-11.409690560974646 26/2/2021 22:47:55
-11.412456514843864 26/2/2021 23:47:55
-11.415222468713084 27/2/2021 00:47:55
-11.417988422582303 27/2/2021 01:47:55
-11.419782523366333 27/2/2021 02:47:55
-11.418390736761372 27/2/2021 03:47:55
-11.416998950156412 27/2/2021 04:47:55
-11.415607163551453 27/2/2021 05:47:55
-11.414215376946494 27/2/2021 06:47:55
-11.412823590341533 27/2/2021 07:47:55
-11.411431803736573 27/2/2021 08:47:55
-11.410040017131614 27/2/2021 09:47:55
-11.410024300099407 27/2/2021 10:47:55
-11.414519191944134 27/2/2021 11:47:55
-11.419014083788873 27/2/2021 12:47:55
-11.4235089756336 27/2/2021 13:47:55
-11.428003867478328 27/2/2021 14:47:55
-11.432498759323053 27/2/2021 15:47:55


## Valores Registrados

In [38]:
# AFData.RecordedValues Method
# RecordedValues(AFTimeRange timeRange, AFBoundaryType boundaryType, UOM desiredUOM, string filterExpression, 
# bool includeFilteredValues, int maxCount = 0)
# boundaryType Type: OSIsoft.AF.Data.AFBoundaryType. A value from the AFBoundaryType enumeration that determines how the 
# times and values of the returned end points are determined.
# Inside: 0 Specifies to return the recorded values on the inside of the requested time range as the first and last values.
# Outside: 1 Specifies to return the recorded values on the outside of the requested time range as the first and last values.
# Interpolated: 2 Specifies to create an interpolated value at the end points of the requested time range if a recorded
# value does not exist at that time.
recorded_values = list(attr[0].Data.RecordedValues(timerange, 0, None, "", False, 100))
for i in range(0, len(recorded_values)-1):
    print(recorded_values[i].Value, recorded_values[i].Timestamp)

-11.397995606487626 26/2/2021 18:34:13
-11.42010784720854 27/2/2021 02:33:53


## Valores Calculados

In [41]:
timerange = AFTimeRange("*-3d", "*")
span = AFTimeSpan.Parse('1d')
summaries = list(attr[0].Data.Summaries(timerange, span, AFSummaryTypes.Average, AFCalculationBasis.TimeWeighted, 
                                        AFTimestampCalculation.Auto))
for i in range(0, len(summaries)-1):
    print(summaries[i].Value, summaries[i].Timestamp)

In [ ]:
nternal static int CountBadData(AFElement element, int depth)
 2{
 3    int count = 0;
 4
 5    // we can use the AFAttributeList to retrieve values from
 6    // multiple attributes at one time.  This is useful when the
 7    // data is coming from a data reference, such as a PI Point, which
 8    // supports retrieving the data in bulk.
 9    AFAttributeList attributes = new AFAttributeList(element.Attributes);
10    AFValues values = attributes.GetValue();
11
12    // count the bad values returned
13    foreach (AFValue v in values)
14    {
15        if (!v.IsGood)
16            count++;
17    }
18
19    // count bad data in child elements too
20    if (depth > 0)
21    {
22        foreach (AFElement childElement in element.Elements)
23        {
24            count += CountBadData(childElement, depth - 1);
25        }
26    }
27
28    return count;

# Pruebas

In [105]:
afservers = PISystems()

In [106]:
#afserver = afservers.DefaultPISystem
AFServer = afservers['PARCCVWPIEDA01']
#AFServer = afservers['QARCCVWPIE01']
print(AFServer.Name)

PARCCVWPIEDA01


In [108]:
# Acceso a una base de datos que no sea la default:
myDB = AFServer.Databases.DefaultDatabase
print(myDB.Name)
myDB = AFServer.Databases.get_Item("DOF")
print ('Database Name: {0}'.format(myDB.Name))

DOF
Database Name: DOF


In [137]:
# Get a specific area of the element hierarchy to load 
search_element = "PAMPA ENERGIA S.A."
myArea = myDB.Elements.get_Item(search_element)
if myArea is None:
    print('Element ', search_element, ' was not found.')

In [138]:
print(myArea.Name)

PAMPA ENERGIA S.A.


# NO OLVIDARSE DE Conectar la VPN

In [139]:
# Use AFElement.LoadToDepth to fully load this part of the hierarchy.
# Note that care needs to be taken to not attempt to load too many objects into memory at once.  Here, we limit the load
# to 3 additional levels of the hierarchy, not to exceed 1000 total elements.
# LoadElementsToDepth(IList<AFElement> elements, bool fullLoad, int depth, int maxCount)
Level1_Elements = AFElement.LoadElementsToDepth(myArea.Elements, True, 1, 1000);

CommunicationException: Cannot connect to server 'PARCCVWPIEDA01'.
   en OSIsoft.AF.Support.AFProxy.Reconnect(AFCollectiveMember member, Boolean autoPrompt, AFConnectionPreference preference)
   en OSIsoft.AF.Support.AFProxy.Reconnect()
   en OSIsoft.AF.Support.AFSerialProxy.Call(String rpcName, ProxyDelegate codeBlock)
   en OSIsoft.AF.Support.AFSerialProxy.LoadElementReferences(Guid[] ids, AFTime queryDate, dcReferenceInformation[]& references)
   en OSIsoft.AF.Asset.AFElement.LoadElementReferences(IList`1 elements)
   en OSIsoft.AF.Asset.AFElement.LoadElementsToDepth(IList`1 elements, Boolean fullLoad, Int32 depth, Int32 maxCount)

In [ ]:
type(Level1_Elements)

In [ ]:
for element in Level1_Elements:
    print(element)

In [ ]:
myPISystem = PISystems.DefaultPISystem

In [20]:
attribute = element.Attributes.get_Item("Nombre Area")

AttributeError: 'NoneType' object has no attribute 'Attributes'

In [ ]:
name = pt.Name.lower()
# CurrentValue
print('\nShowing PI Tag CurrentValue from {0}'.format(name))
current_value = pt.CurrentValue()
print( '{0}\'s Current Value: {1}'.format(name, current_value.Value))

In [ ]:
#recordedvalues
timerange = AFTimeRange("*-3h", "*")
recorded = pt.RecordedValues(timerange, AFBoundaryType.Inside, "", False)
print('\nShowing PI Tag RecordedValues from {0}'.format(name))
for event in recorded:
    print('{0} value: {1}'.format(event.Timestamp.LocalTime, event.Value))

In [ ]:
#plotValues
plotvalues = pt.PlotValues(timerange, 100)
print('\nShowing PI Tag PlotValues from {0}'.format(name))
for event in plotvalues:
    print('{0} value: {1}'.format(event.Timestamp.LocalTime, event.Value))

In [ ]:
#interpolatedvalues
span = AFTimeSpan.Parse("1h")
interpolated = pt.InterpolatedValues(timerange, span, "", False)
print('\nShowing PI Tag InterpolatedValues from {0}'.format(name))
for event in interpolated:
    print('{0} value: {1}'.format(event.Timestamp.LocalTime, event.Value))

In [ ]:
#summariesvalues
summaries = pt.Summaries(timerange, span, AFSummaryTypes.Average, AFCalculationBasis.TimeWeighted, AFTimestampCalculation.Auto)
print('\nShowing PI Tag SummariesValues(Average) from {0}'.format(name))
for summary in summaries:
    for event in summary.Value:
        print('{0} value: {1}'.format(event.Timestamp.LocalTime, event.Value))

In [ ]:
#writeValue
writept = PIPoint.FindPIPoint(piServer,"PleaseEnterWriteTagName")
writeptname = writept.Name.lower()
val = AFValue()
val.Value = 20
#val.Timestamp = AFTime("t+9h")

In [ ]:
print('\nWrite value to {0} value: {1}'.format(writeptname, val.Value))
writept.UpdateValue(val, AFUpdateOption.Replace, AFBufferOption.BufferIfPossible)

In [ ]:
#Connect to AF
print ('\nConnect to AF')
afServers = PISystems()
afServer = afServers.DefaultPISystem
DB = afServer.Databases.DefaultDatabase
#DB = afServer.Databases.get_Item("NuGreen")
element = DB.Elements.get_Item("NuGreen").Elements.get_Item("Little Rock").\
    Elements.get_Item("Extruding Process").Elements.get_Item("Equipment").Elements.get_Item("K-435")

In [ ]:
attribute = element.Attributes.get_Item("Steam Flow")
attval = attribute.GetValue()

In [ ]:
print ('Element Name: {0}'.format(element.Name))
print ('Attribute Name: {0} | Value : {1} {2}'.format(attribute.Name, attval.Value, attribute.DefaultUOM))

In [ ]:
#create element with attribute
print('\nCreate Element with Attribute')
if DB.Elements.get_Item("Test New Element") is not None:
    print("Already Existing Element: Test New Element")
else:
    newelement = DB.Elements.Add("Test New Element")
    newelement.Description = "Created element from PIthon"
    newattribute = newelement.Attributes.Add("Test Attribute")
    newattribute.DataReferencePlugIn = afServer.DataReferencePlugIns.get_Item("PI Point")
    newattribute.DataReference.ConfigString = "cdt158"
    DB.CheckIn()
    print("Created new Element : Test New Element")